In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats

# 1. Load the dataset
# Replace with your actual file path
file_path = "C:/Users/bazrafka/Desktop/counting/DiscussionPaperData/forLovepreet2.xlsx" 
df = pd.read_excel(file_path)

# 2. Setup Factors
# Ensure ENTRY, Location, and REP are treated as categorical factors
df['ENTRY'] = df['ENTRY'].astype('category')
df['Location'] = df['Location'].astype('category')
df['REP'] = df['REP'].astype('category')

# List of traits you want to analyze
traits = ['CWT_A', 'CorrectedYield']

# Prepare an Excel writer to save all results in one file
with pd.ExcelWriter("Agricultural_Analysis_Summary.xlsx") as writer:
    
    for trait in traits:
        print(f"--- Processing Trait: {trait} ---")
        
        # 3. Perform ANOVA
        # Formula: Trait ~ ENTRY * Location + REP
        formula = f"{trait} ~ C_ENTRY * C_Location + C_REP"
        # Renaming temporarily for statsmodels compatibility (factors denoted by C())
        model_data = df.rename(columns={'ENTRY': 'C_ENTRY', 'Location': 'C_Location', 'REP': 'C_REP'})
        
        model = ols(formula, data=model_data).fit()
        anova_table = sm.stats.anova_lm(model, typ=2)
        print(anova_table)

        # 4. Calculate LSD manually (matching your R logic)
        # Extract Mean Square Error (Residuals) and Degrees of Freedom
        mse = anova_table.loc['Residual', 'sum_sq'] / anova_table.loc['Residual', 'df']
        df_error = anova_table.loc['Residual', 'df']
        
        # Harmonic mean of sample sizes per ENTRY
        sample_sizes = df.groupby('ENTRY').size()
        harmonic_mean_n = len(sample_sizes) / np.sum(1.0 / sample_sizes)
        
        # Critical t-value at 95% confidence (alpha = 0.05)
        alpha = 0.05
        t_crit = stats.t.ppf(1 - alpha/2, df_error)
        
        # Compute LSD Value
        lsd_value = t_crit * np.sqrt(2 * mse / harmonic_mean_n)
        
        # 5. Create Summary Dataframe (Means by ENTRY)
        summary_df = df.groupby('ENTRY')[trait].mean().reset_index()
        summary_df.columns = ['group', 'means']
        summary_df['LSD_Threshold'] = lsd_value
        
        # Note: True grouping letters (a, b, c) usually require the 'bioinfokit' 
        # or manual implementation. For now, we provide the means and the LSD.
        
        # 6. Save to separate sheet in Excel
        summary_df.to_excel(writer, sheet_name=trait, index=False)

print("\nAnalysis complete! Results saved to 'Agricultural_Analysis_Summary.xlsx'")

--- Processing Trait: CWT_A ---


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 261, but rank is 127
  warnings.warn('covariance of constraints does not have full '


                          sum_sq      df          F        PR(>F)
C_ENTRY             17934.519980   261.0   2.388770  2.989245e-15
C_Location           2278.672297     1.0  79.214899  1.014813e-18
C_REP                 492.140920     3.0   5.702867  6.881104e-04
C_ENTRY:C_Location   9432.124199   261.0   1.256302  4.809371e-03
Residual            75625.034127  2629.0        NaN           NaN
--- Processing Trait: CorrectedYield ---


C:\Users\bazrafka\AppData\Local\Temp\ipykernel_62668\1598798821.py:43: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sample_sizes = df.groupby('ENTRY').size()
C:\Users\bazrafka\AppData\Local\Temp\ipykernel_62668\1598798821.py:54: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summary_df = df.groupby('ENTRY')[trait].mean().reset_index()
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 261, but rank is 127
  warnings.warn('covariance of constraints does not have

                          sum_sq      df          F        PR(>F)
C_ENTRY             16806.664635   261.0   2.250837  2.910939e-13
C_Location           1577.658632     1.0  55.146262  1.504200e-13
C_REP                 659.949963     3.0   7.689406  4.093292e-05
C_ENTRY:C_Location   7664.652220   261.0   1.026491  3.779584e-01
Residual            75097.636609  2625.0        NaN           NaN

Analysis complete! Results saved to 'Agricultural_Analysis_Summary.xlsx'


C:\Users\bazrafka\AppData\Local\Temp\ipykernel_62668\1598798821.py:43: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sample_sizes = df.groupby('ENTRY').size()
C:\Users\bazrafka\AppData\Local\Temp\ipykernel_62668\1598798821.py:54: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summary_df = df.groupby('ENTRY')[trait].mean().reset_index()


In [22]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols

# 1. Load data
#file_path = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\forLovepreet2.xlsx"
file_path = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\mrc22.xlsx"

df = pd.read_excel(file_path)

# Ensure factors are categorical
for col in ['ENTRY', 'Location', 'REP', 'ExperimentName']:
    df[col] = df[col].astype('category')

traits = ['CWT_A', 'CorrectedYield']
results_list = []

# 2. Group by Experiment Name
for exp_name, group_data in df.groupby('ExperimentName'):
    # Skip if group is too small to run ANOVA
    if len(group_data) < 10:
        continue
        
    for trait in traits:
        try:
            # 3. Fit ANOVA for this specific Experiment
            # Formula: Trait ~ ENTRY + Location + REP (Simplified if only 1 location)
            formula = f"Q('{trait}') ~ C(ENTRY) + C(Location) + C(REP)"
            model = ols(formula, data=group_data).fit()
            anova_table = sm.stats.anova_lm(model, typ=2)
            
            # 4. Extract Statistics
            mse = anova_table.loc['Residual', 'sum_sq'] / anova_table.loc['Residual', 'df']
            grand_mean = group_data[trait].mean()
            
            # 5. Calculate %CV
            cv_value = (np.sqrt(mse) / grand_mean) * 100
            
            # 6. Store results
            results_list.append({
                'Experiment': exp_name,
                'Trait': trait,
                'Grand_Mean': round(grand_mean, 2),
                'MSE': round(mse, 4),
                'Percent_CV': round(cv_value, 2)
            })
        except Exception as e:
            print(f"Could not process {exp_name} for {trait}: {e}")

# 7. Create Summary Table and Save
cv_summary = pd.DataFrame(results_list)
cv_summary.to_excel("Experiment_CV_Results.xlsx", index=False)
print(cv_summary)

Could not process 2217 for CWT_A: must have at least one row in constraint matrix
Could not process 2217 for CorrectedYield: must have at least one row in constraint matrix
Could not process 2218 for CWT_A: must have at least one row in constraint matrix
Could not process 2218 for CorrectedYield: must have at least one row in constraint matrix
Could not process 2219 for CWT_A: must have at least one row in constraint matrix


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 127, but rank is 15
  warnings.warn('covariance of constraints does not have full '
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 127, but rank is 15
  warnings.warn('covariance of constraints does not have full '
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 127, but rank is 23
  warnings.warn('covariance of constraints does not have full '


Could not process 2219 for CorrectedYield: must have at least one row in constraint matrix
Empty DataFrame
Columns: []
Index: []


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 127, but rank is 23
  warnings.warn('covariance of constraints does not have full '


In [ ]:
#first remove outlier

In [23]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols

# --- OUTLIER REMOVAL FUNCTION ---
def remove_outliers(data, column):
    """Removes outliers based on the 1.5 * IQR rule."""
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # Keep only data within bounds
    return data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]

# 1. Load data
file_path = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\mrc22.xlsx"
df = pd.read_excel(file_path)

# Ensure factors are categorical
for col in ['ENTRY', 'Location', 'REP', 'ExperimentName']:
    df[col] = df[col].astype('category')

traits = ['CWT_A', 'CorrectedYield']
results_list = []

# 2. Group by Experiment Name
for exp_name, group_data in df.groupby('ExperimentName'):
    if len(group_data) < 10:
        continue
        
    for trait in traits:
        try:
            # --- STEP: REMOVE OUTLIERS FOR THIS SPECIFIC TRAIT/EXPERIMENT ---
            # We clean the data specifically for the trait we are about to analyze
            clean_data = remove_outliers(group_data.copy(), trait)
            
            # 3. Fit ANOVA for this specific Experiment using CLEANED data
            formula = f"Q('{trait}') ~ C(ENTRY) + C(Location) + C(REP)"
            model = ols(formula, data=clean_data).fit()
            anova_table = sm.stats.anova_lm(model, typ=2)
            
            # 4. Extract Statistics
            mse = anova_table.loc['Residual', 'sum_sq'] / anova_table.loc['Residual', 'df']
            grand_mean = clean_data[trait].mean()
            
            # 5. Calculate %CV
            cv_value = (np.sqrt(mse) / grand_mean) * 100
            
            # 6. Store results
            results_list.append({
                'Experiment': exp_name,
                'Trait': trait,
                'Original_Count': len(group_data),
                'Cleaned_Count': len(clean_data),
                'Grand_Mean': round(grand_mean, 2),
                'MSE': round(mse, 4),
                'Percent_CV': round(cv_value, 2)
            })
        except Exception as e:
            print(f"Could not process {exp_name} for {trait}: {e}")

# 7. Create Summary Table and Save
cv_summary = pd.DataFrame(results_list)
cv_summary.to_excel("Experiment_CV_Results_Cleaned.xlsx", index=False)
print(cv_summary)

Could not process 2217 for CWT_A: must have at least one row in constraint matrix
Could not process 2217 for CorrectedYield: must have at least one row in constraint matrix
Could not process 2218 for CWT_A: must have at least one row in constraint matrix
Could not process 2218 for CorrectedYield: must have at least one row in constraint matrix
Could not process 2219 for CWT_A: must have at least one row in constraint matrix


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 127, but rank is 15
  warnings.warn('covariance of constraints does not have full '
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 127, but rank is 15
  warnings.warn('covariance of constraints does not have full '
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 127, but rank is 23
  warnings.warn('covariance of constraints does not have full '


Could not process 2219 for CorrectedYield: must have at least one row in constraint matrix
Empty DataFrame
Columns: []
Index: []


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 127, but rank is 23
  warnings.warn('covariance of constraints does not have full '


In [39]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

def remove_outliers(data, column):
    if data[column].isnull().all():
        return data
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    return data[(data[column] >= (Q1 - 1.5 * IQR)) & (data[column] <= (Q3 + 1.5 * IQR))]

#file_path = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\mrc22.xlsx"
file_path = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Outputs\MRC2022_\EXCEL\Joined\All_Trials_Combined.xlsx"

df = pd.read_excel(file_path)

for col in ['ENTRY', 'Location', 'REP', 'ExperimentName']:
    if col in df.columns:
        df[col] = df[col].astype('category')

traits = ['CWT_A', 'CorrectedYield']
results_list = []

for exp_name, group_data in df.groupby('ExperimentName', observed=False):
    if len(group_data) < 5: # Lowered threshold to see if we get anything
        continue
        
    for trait in traits:
        if trait not in group_data.columns:
            continue
            
        try:
            clean_data = remove_outliers(group_data.copy(), trait)
            
            # --- DYNAMIC FORMULA BUILDING ---
            # Start with ENTRY, add others ONLY if they have more than 1 level
            formula_parts = ["C(ENTRY)"]
            
            if clean_data['Location'].nunique() > 1:
                formula_parts.append("C(Location)")
            if clean_data['REP'].nunique() > 1:
                formula_parts.append("C(REP)")
            
            formula = f"Q('{trait}') ~ " + " + ".join(formula_parts)
            
            # Fit model
            model = ols(formula, data=clean_data).fit()
            anova_table = sm.stats.anova_lm(model, typ=2)
            
            mse = anova_table.loc['Residual', 'sum_sq'] / anova_table.loc['Residual', 'df']
            grand_mean = clean_data[trait].mean()
            cv_value = (np.sqrt(mse) / grand_mean) * 100
            
            results_list.append({
                'Experiment': exp_name,
                'Trait': trait,
                'Percent_CV': round(cv_value, 2),
                'Grand_Mean': round(grand_mean, 2),
                'Formula_Used': formula
            })
            
        except Exception as e:
            print(f"Skipping {exp_name} for {trait}: {e}")

if results_list:
    cv_summary = pd.DataFrame(results_list)
    cv_summary.to_excel("Experiment_CV_Results_Simplified.xlsx", index=False)
    print("\nSuccess! Results saved.")
else:
    print("\nStill no results. Check if CWT_A and CorrectedYield contain numbers or if they are all empty.")


Success! Results saved.


In [38]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import warnings

# Suppress technical warnings for cleaner output
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore', message='covariance of constraints does not have full rank')
warnings.filterwarnings('ignore', category=sm.tools.sm_exceptions.ConvergenceWarning)

# --- OUTLIER REMOVAL FUNCTION ---
def remove_outliers(data, column):
    """Removes outliers based on the 1.5 * IQR rule."""
    if data[column].isnull().all():
        return data
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return data[(data[column] >= lower) & (data[column] <= upper)]

# 1. Load data
file_path = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Outputs\MRC2022_\EXCEL\Joined\All_Trials_Combined.xlsx"

#file_path = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Outputs\2021_\excel\Joined\All_Trials_Combined.xlsx"

df = pd.read_excel(file_path)

# Ensure factors are categorical
# Using 'IBLK' for incomplete blocks to reduce CV
for col in ['ENTRY', 'Location', 'REP', 'IBLK', 'ExperimentName']:
    if col in df.columns:
        df[col] = df[col].astype('category')

traits = ['CWT_A', 'CorrectedYield']
results_list = []
CLEAN_OUTLIERS = False  # Set to True if you want to remove outliers first

# 2. Group by Experiment Name
for exp_name, group_data in df.groupby('ExperimentName', observed=False):
    if len(group_data) < 10: 
        continue
        
    for trait in traits:
        if trait not in group_data.columns:
            continue
            
        try:
            # Prepare data
            analysis_data = remove_outliers(group_data.copy(), trait) if CLEAN_OUTLIERS else group_data.copy()
            
            # Check for variation
            if analysis_data[trait].std() == 0 or analysis_data[trait].isnull().all():
                continue

            # --- DYNAMIC FORMULA BUILDING ---
            fixed_parts = ["C(ENTRY)"]
            if analysis_data['Location'].nunique() > 1:
                fixed_parts.append("C(Location)")
            if analysis_data['REP'].nunique() > 1:
                fixed_parts.append("C(REP)")
            
            fixed_formula = f"Q('{trait}') ~ " + " + ".join(fixed_parts)
            
            # 3. Fit Mixed Linear Model (REML)
            # We use 'lbfgs' and high maxiter to help with ConvergenceWarnings
            model = smf.mixedlm(fixed_formula, analysis_data, groups=analysis_data["REP"], re_formula="~0 + C(IBLK)")
            result = model.fit(method=["lbfgs"], maxiter=20000)
            
            # 4. Extract Statistics for %CV
            # mse = result.scale (Residual Variance)
            mse = result.scale 
            grand_mean = analysis_data[trait].mean()
            cv_value = (np.sqrt(mse) / grand_mean) * 100
            
            results_list.append({
                'Experiment': exp_name,
                'Trait': trait,
                'Percent_CV': round(cv_value, 2),
                'Grand_Mean': round(grand_mean, 2),
                'MSE': round(mse, 4),
                'Model_Type': "Mixed Model (Random IBLK)"
            })
            
        except Exception as e:
            # FALLBACK: If Mixed Model fails to converge/run, use standard OLS
            try:
                formula_ols = f"Q('{trait}') ~ C(ENTRY) + C(REP)"
                model_ols = smf.ols(formula_ols, data=analysis_data).fit()
                mse_ols = model_ols.mse_resid
                cv_ols = (np.sqrt(mse_ols) / analysis_data[trait].mean()) * 100
                
                results_list.append({
                    'Experiment': exp_name,
                    'Trait': trait,
                    'Percent_CV': round(cv_ols, 2),
                    'Grand_Mean': round(analysis_data[trait].mean(), 2),
                    'MSE': round(mse_ols, 4),
                    'Model_Type': "OLS Fallback"
                })
            except:
                print(f"Skipping {exp_name} for {trait}: {e}")

# 5. Save Summary
if results_list:
    cv_summary = pd.DataFrame(results_list)
    cv_summary.to_excel("Experiment_CV_MaxedModel_Final.xlsx", index=False)
    print("\nSuccess! Analysis complete.")
    print(cv_summary.head(10))
else:
    print("\nNo results generated. Check column names.")

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:1634: UserWarning: Random effects covariance is singular
  warnings.warn(msg)



Success! Analysis complete.
   Experiment           Trait  Percent_CV  Grand_Mean      MSE  \
0        2217           CWT_A       16.34       19.92  10.5948   
1        2217  CorrectedYield       16.99       15.97   7.3615   
2        2218           CWT_A       16.12       21.43  11.9370   
3        2218  CorrectedYield       16.38       17.15   7.8913   
4        2219           CWT_A       13.83       19.66   7.3923   
5        2219  CorrectedYield       13.72       15.08   4.2842   

                  Model_Type  
0  Mixed Model (Random IBLK)  
1  Mixed Model (Random IBLK)  
2               OLS Fallback  
3               OLS Fallback  
4               OLS Fallback  
5               OLS Fallback  


In [ ]:
#To determine which part of the data affects the overall Coefficient of Variation (CV) more,
#we need to split your data into "Low," "Medium," and "High" yield tiers (quantiles) and run 
#the Mixed Model analysis on each subset separately.

#The following code calculates the local CV for these three tiers. 
#If the "Low" tier has a significantly higher CV than the "High" tier, it suggests that measurement errors or
#plot variability at lower yield levels are the primary drivers of your overall noise.

In [4]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import warnings

# Suppress technical warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore', message='covariance of constraints does not have full rank')
warnings.filterwarnings('ignore', category=sm.tools.sm_exceptions.ConvergenceWarning)

# --- OUTLIER REMOVAL FUNCTION ---
def remove_outliers(data, column):
    if data[column].isnull().all():
        return data
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return data[(data[column] >= lower) & (data[column] <= upper)]

# 1. Load data
#file_path = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Outputs\MRC2022_\EXCEL\Joined\All_Trials_Combined.xlsx"
#file_path = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Outputs\2024_\excell\Joined\All_Trials_Combined.xlsx"
#file_path = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Outputs\SEVREC2022_\excel\Joined\All_Trials_Combined.xlsx"
#file_path = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Outputs\sevrec25_\excel\Joined\All_Trials_Combined.xlsx"
#file_path = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Outputs\2021_\excel\Joined\All_Trials_Combined.xlsx"
#file_path = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Outputs\AllYearsLocationsExceptCanadaNEW.xlsx"
#file_path = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Outputs\Canada25\excel\Joined\All_Trials_Combined.xlsx"
file_path = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Outputs\Canada24\excel\Joined\All_Trials_Combined.xlsx"





df = pd.read_excel(file_path)

for col in ['ENTRY', 'Location', 'REP', 'IBLK', 'Experiment Name']:
    if col in df.columns:
        df[col] = df[col].astype('category')

traits = ['CWT_A', 'CorrectedYield']
results_list = []
CLEAN_OUTLIERS = False 

# 2. Group by Experiment Name
for exp_name, group_data in df.groupby('Experiment Name', observed=False):
    if len(group_data) < 15: # Increased threshold to ensure enough data for tiers
        continue
        
    for trait in traits:
        if trait not in group_data.columns:
            continue
            
        analysis_data = remove_outliers(group_data.copy(), trait) if CLEAN_OUTLIERS else group_data.copy()
        
        # --- NEW STEP: SPLIT INTO TIERS ---
        # We define Low (Bottom 33%), Medium (33-66%), and High (Top 33%) yield tiers
        try:
            analysis_data['Yield_Tier'] = pd.qcut(analysis_data[trait], 3, labels=["Low", "Medium", "High"])
        except ValueError: # If data is too uniform to split
            continue

        for tier in ["Low", "Medium", "High"]:
            tier_data = analysis_data[analysis_data['Yield_Tier'] == tier].copy()
            
            if len(tier_data) < 5: continue # Skip if tier is too small
            
            try:
                # Dynamic Formula
                fixed_parts = ["C(ENTRY)"]
                if tier_data['REP'].nunique() > 1:
                    fixed_parts.append("C(REP)")
                
                fixed_formula = f"Q('{trait}') ~ " + " + ".join(fixed_parts)
                
                # Fit Mixed Model
                model = smf.mixedlm(fixed_formula, tier_data, groups=tier_data["REP"], re_formula="~0 + C(IBLK)")
                result = model.fit(method=["lbfgs"], maxiter=20000)
                
                mse = result.scale 
                mean_val = tier_data[trait].mean()
                cv_value = (np.sqrt(mse) / mean_val) * 100
                
                results_list.append({
                    'Experiment': exp_name,
                    'Trait': trait,
                    'Tier': tier,
                    'Percent_CV': round(cv_value, 2),
                    'Mean_Yield': round(mean_val, 2),
                    'MSE': round(mse, 4)
                })
                
            except Exception:
                # OLS Fallback for Tiers
                try:
                    formula_ols = f"Q('{trait}') ~ C(ENTRY)"
                    model_ols = smf.ols(formula_ols, data=tier_data).fit()
                    mse_ols = model_ols.mse_resid
                    cv_ols = (np.sqrt(mse_ols) / tier_data[trait].mean()) * 100
                    
                    results_list.append({
                        'Experiment': exp_name,
                        'Trait': trait,
                        'Tier': tier,
                        'Percent_CV': round(cv_ols, 2),
                        'Mean_Yield': round(tier_data[trait].mean(), 2),
                        'MSE': round(mse_ols, 4)
                    })
                except:
                    continue

# 3. Save and Summarize
if results_list:
    tier_summary = pd.DataFrame(results_list)
    tier_summary.to_excel("CV_Yield_Tier_Analysis.xlsx", index=False)
    
    # Calculate group averages to see the trend
    trend = tier_summary.groupby(['Trait', 'Tier'])['Percent_CV'].mean().unstack()
    print("\n--- Average CV by Yield Tier ---")
    print(trend)
    print("\nAnalysis complete. Check 'CV_Yield_Tier_Analysis.xlsx' for details.")


--- Average CV by Yield Tier ---
Tier             High    Low  Medium
Trait                               
CWT_A           10.04  30.89    5.94
CorrectedYield  15.83  21.09    6.43

Analysis complete. Check 'CV_Yield_Tier_Analysis.xlsx' for details.
